Instalación de Dependencias



In [2]:
# Celda 1: Instalación de dependencias necesarias
print("🔧 Instalando todas las dependencias...")
!pip install sentence-transformers langchain langchain-community pypdf2 supabase tiktoken tqdm numpy

print("\n✅ Proceso completado. Los conflictos de dependencias deberían estar resueltos.")

🔧 Instalando todas las dependencias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.

✅ Proceso completado. Los conflictos de dependencias deberían estar resueltos.


Importar librerías

In [3]:
# Celda 2: Importar librerías
import os
import json
import hashlib
from typing import List, Dict, Tuple
import numpy as np
from tqdm import tqdm
from datetime import datetime
from google.colab import userdata, files
import warnings
warnings.filterwarnings('ignore')

# Librerías para procesamiento
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# Supabase
from supabase import create_client, Client

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


Cargar secretos de forma segura


In [4]:
# Celda 3: Cargar secretos de forma segura
try:
    # Obtener secretos desde Google Colab
    SUPABASE_URL = userdata.get('SUPABASE_URL')
    SUPABASE_KEY = userdata.get('SUPABASE_KEY')

    # Validar que los secretos existen
    if not SUPABASE_URL or not SUPABASE_KEY:
        raise ValueError("Secretos no configurados")

    print("✅ Secretos cargados correctamente")
    print(f"📍 Supabase URL: {SUPABASE_URL[:20]}...")

except Exception as e:
    print("❌ Error: No se encontraron los secretos de Colab")
    print("Por favor, configura los siguientes secretos en Colab:")
    print("1. Click en el ícono 🔑 en el panel izquierdo")
    print("2. Agrega SUPABASE_URL y SUPABASE_KEY")
    raise e

✅ Secretos cargados correctamente
📍 Supabase URL: https://jqhmaqjzszaq...


Mostrar SQL necesario para Supabase


In [4]:
sql_setup = """
-- Crear extensión para vectores
CREATE EXTENSION IF NOT EXISTS vector;

-- Crear tabla para documentos
CREATE TABLE IF NOT EXISTS mapstruct_documents (
    id SERIAL PRIMARY KEY,
    content TEXT NOT NULL,
    embedding vector(384),
    metadata JSONB,
    chunk_hash VARCHAR(64) UNIQUE,
    created_at TIMESTAMP DEFAULT NOW()
);

-- Crear índice para búsqueda vectorial eficiente
CREATE INDEX IF NOT EXISTS idx_mapstruct_embedding
ON mapstruct_documents
USING ivfflat (embedding vector_cosine_ops)
WITH (lists = 100);

-- Función para búsqueda de similitud
CREATE OR REPLACE FUNCTION match_mapstruct_documents (
    query_embedding vector(384),
    match_threshold float,
    match_count int
)
RETURNS TABLE (
    id int,
    content text,
    metadata jsonb,
    similarity float
)
LANGUAGE sql STABLE
AS $$
    SELECT
        id,
        content,
        metadata,
        1 - (embedding <=> query_embedding) as similarity
    FROM mapstruct_documents
    WHERE 1 - (embedding <=> query_embedding) > match_threshold
    ORDER BY embedding <=> query_embedding
    LIMIT match_count;
$$;
"""

print("📋 SQL para ejecutar en Supabase Dashboard:")
print("=" * 50)
print(sql_setup)
print("=" * 50)
print("\n⚠️ Asegúrate de ejecutar este SQL en tu dashboard de Supabase antes de continuar")


📋 SQL para ejecutar en Supabase Dashboard:

-- Crear extensión para vectores
CREATE EXTENSION IF NOT EXISTS vector;

-- Crear tabla para documentos
CREATE TABLE IF NOT EXISTS mapstruct_documents (
    id SERIAL PRIMARY KEY,
    content TEXT NOT NULL,
    embedding vector(384),
    metadata JSONB,
    chunk_hash VARCHAR(64) UNIQUE,
    created_at TIMESTAMP DEFAULT NOW()
);

-- Crear índice para búsqueda vectorial eficiente
CREATE INDEX IF NOT EXISTS idx_mapstruct_embedding 
ON mapstruct_documents 
USING ivfflat (embedding vector_cosine_ops)
WITH (lists = 100);

-- Función para búsqueda de similitud
CREATE OR REPLACE FUNCTION match_mapstruct_documents (
    query_embedding vector(384),
    match_threshold float,
    match_count int
)
RETURNS TABLE (
    id int,
    content text,
    metadata jsonb,
    similarity float
)
LANGUAGE sql STABLE
AS $$
    SELECT 
        id,
        content,
        metadata,
        1 - (embedding <=> query_embedding) as similarity
    FROM mapstruct_documen

Función para cargar archivos


In [5]:
# Celda 5: Función para cargar archivos
def upload_documents():
    """Permite cargar archivos PDF y TXT desde tu computadora"""
    print("📤 Por favor, selecciona tus archivos PDF y/o TXT")
    uploaded = files.upload()

    documents = []
    for filename, content in uploaded.items():
        doc_info = {
            'filename': filename,
            'content': content,
            'type': 'pdf' if filename.endswith('.pdf') else 'txt'
        }
        documents.append(doc_info)
        print(f"✅ Archivo cargado: {filename} ({len(content):,} bytes)")

    return documents

# Cargar documentos
print("🔄 Iniciando carga de documentos...")
uploaded_docs = upload_documents()

🔄 Iniciando carga de documentos...
📤 Por favor, selecciona tus archivos PDF y/o TXT


Saving mapstruct-reference-guide.pdf to mapstruct-reference-guide.pdf
Saving ASO  Platform Documentation 2025.txt to ASO  Platform Documentation 2025.txt
✅ Archivo cargado: mapstruct-reference-guide.pdf (2,629,818 bytes)
✅ Archivo cargado: ASO  Platform Documentation 2025.txt (671,480 bytes)


Funciones de procesamiento de documentos


In [6]:
# Celda 6: Funciones de procesamiento de documentos
class DocumentProcessor:
    """Procesador optimizado para documentos técnicos"""

    def __init__(self, chunk_size=1000, chunk_overlap=200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            separators=[
                "\n\n",      # Párrafos
                "\n",        # Líneas
                ".",         # Oraciones
                ";",         # Punto y coma
                ",",         # Comas
                " ",         # Espacios
                ""           # Caracteres individuales
            ]
        )

    def process_pdf(self, content: bytes, filename: str) -> str:
        """Extrae texto de un PDF con manejo de errores mejorado"""
        try:
            # Guardar temporalmente el archivo
            temp_path = f"/tmp/{filename}"
            with open(temp_path, 'wb') as f:
                f.write(content)

            # Leer el PDF
            text = ""
            with open(temp_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                total_pages = len(pdf_reader.pages)

                print(f"📖 Procesando {total_pages} páginas de {filename}")

                for page_num in tqdm(range(total_pages), desc="Páginas"):
                    page = pdf_reader.pages[page_num]
                    page_text = page.extract_text()

                    # Limpiar el texto
                    page_text = self.clean_text(page_text)
                    text += page_text + "\n\n"

            # Eliminar archivo temporal
            os.remove(temp_path)
            return text

        except Exception as e:
            print(f"❌ Error procesando PDF {filename}: {str(e)}")
            return ""

    def process_txt(self, content: bytes, filename: str) -> str:
        """Procesa archivo de texto plano"""
        try:
            text = content.decode('utf-8')
            return self.clean_text(text)
        except Exception as e:
            print(f"❌ Error procesando TXT {filename}: {str(e)}")
            return ""

    def clean_text(self, text: str) -> str:
        """Limpia y normaliza el texto"""
        # Eliminar espacios múltiples
        text = ' '.join(text.split())

        # Eliminar caracteres especiales problemáticos
        text = text.replace('\x00', '')
        text = text.replace('\r\n', '\n')

        # Mantener estructura de párrafos
        text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())

        return text

    def create_chunks(self, text: str, metadata: Dict) -> List[Dict]:
        """Crea chunks con metadatos enriquecidos"""
        chunks = self.text_splitter.split_text(text)

        chunk_docs = []
        for i, chunk in enumerate(chunks):
            # Generar hash único para evitar duplicados
            chunk_hash = hashlib.sha256(chunk.encode()).hexdigest()

            chunk_doc = {
                'content': chunk,
                'metadata': {
                    **metadata,
                    'chunk_index': i,
                    'chunk_size': len(chunk),
                    'total_chunks': len(chunks)
                },
                'chunk_hash': chunk_hash
            }
            chunk_docs.append(chunk_doc)

        return chunk_docs

# Inicializar procesador
processor = DocumentProcessor(chunk_size=1000, chunk_overlap=200)
print("✅ Procesador de documentos inicializado")

✅ Procesador de documentos inicializado


Procesar todos los documentos cargados


In [7]:
# Celda 7: Procesar todos los documentos cargados
all_chunks = []

for doc in uploaded_docs:
    print(f"\n🔄 Procesando: {doc['filename']}")

    # Extraer texto según el tipo de archivo
    if doc['type'] == 'pdf':
        text = processor.process_pdf(doc['content'], doc['filename'])
    else:
        text = processor.process_txt(doc['content'], doc['filename'])

    if text:
        # Crear metadatos base
        metadata = {
            'source': doc['filename'],
            'type': doc['type'],
            'processed_at': datetime.now().isoformat()
        }

        # Crear chunks
        chunks = processor.create_chunks(text, metadata)
        all_chunks.extend(chunks)

        print(f"✅ Generados {len(chunks)} chunks de {doc['filename']}")
        print(f"   Caracteres totales: {len(text):,}")
    else:
        print(f"⚠️ No se pudo extraer texto de {doc['filename']}")

print(f"\n📊 Resumen:")
print(f"Total de chunks generados: {len(all_chunks)}")
print(f"Tamaño promedio de chunk: {np.mean([len(c['content']) for c in all_chunks]):.0f} caracteres")


🔄 Procesando: mapstruct-reference-guide.pdf
📖 Procesando 114 páginas de mapstruct-reference-guide.pdf


Páginas: 100%|██████████| 114/114 [00:02<00:00, 50.66it/s]


✅ Generados 234 chunks de mapstruct-reference-guide.pdf
   Caracteres totales: 167,378

🔄 Procesando: ASO  Platform Documentation 2025.txt
✅ Generados 793 chunks de ASO  Platform Documentation 2025.txt
   Caracteres totales: 619,883

📊 Resumen:
Total de chunks generados: 1027
Tamaño promedio de chunk: 868 caracteres


Configurar y cargar modelo de embeddings

In [8]:
# Celda 8: Configurar y cargar modelo de embeddings
print("🤖 Cargando modelo de embeddings...")
print("Modelo: all-MiniLM-L6-v2")
print("Dimensiones: 384")
print("Optimizado para: Búsqueda semántica")

# Cargar modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Información del modelo
print("\n📊 Información del modelo:")
print(f"Max sequence length: {model.max_seq_length}")
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")
print("✅ Modelo cargado correctamente")

🤖 Cargando modelo de embeddings...
Modelo: all-MiniLM-L6-v2
Dimensiones: 384
Optimizado para: Búsqueda semántica


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📊 Información del modelo:
Max sequence length: 256
Embedding dimension: 384
✅ Modelo cargado correctamente


Generar embeddings para todos los chunks


In [9]:
# Celda 9: Generar embeddings para todos los chunks
def generate_embeddings_batch(chunks: List[Dict], model, batch_size: int = 32) -> List[Dict]:
    """Genera embeddings en lotes para mejor rendimiento"""

    # Preparar textos para embedding
    texts = [chunk['content'] for chunk in chunks]

    # Generar embeddings en lotes
    print(f"🔄 Generando embeddings para {len(texts)} chunks...")
    embeddings = []

    for i in tqdm(range(0, len(texts), batch_size), desc="Lotes"):
        batch_texts = texts[i:i + batch_size]
        batch_embeddings = model.encode(
            batch_texts,
            normalize_embeddings=True,  # Normalizar para cosine similarity
            show_progress_bar=False
        )
        embeddings.extend(batch_embeddings)

    # Agregar embeddings a los chunks
    for chunk, embedding in zip(chunks, embeddings):
        chunk['embedding'] = embedding.tolist()

    return chunks

# Generar embeddings
chunks_with_embeddings = generate_embeddings_batch(all_chunks, model, batch_size=32)

print(f"✅ Embeddings generados para {len(chunks_with_embeddings)} chunks")
print(f"📐 Dimensión de embeddings: {len(chunks_with_embeddings[0]['embedding'])}")

🔄 Generando embeddings para 1027 chunks...


Lotes: 100%|██████████| 33/33 [00:03<00:00,  9.77it/s]

✅ Embeddings generados para 1027 chunks
📐 Dimensión de embeddings: 384


Conectar con Supabase y crear funciones de almacenamiento


In [10]:
# Celda 10: Conectar con Supabase y crear funciones de almacenamiento
class SupabaseVectorStore:
    """Gestiona el almacenamiento de vectores en Supabase"""

    def __init__(self, url: str, key: str):
        self.client = create_client(url, key)
        print("✅ Conectado a Supabase")

    def insert_documents(self, documents: List[Dict], batch_size: int = 50):
        """Inserta documentos en lotes con manejo de duplicados"""

        total = len(documents)
        inserted = 0
        duplicates = 0
        errors = 0

        print(f"🔄 Insertando {total} documentos en Supabase...")

        for i in tqdm(range(0, total, batch_size), desc="Insertando"):
            batch = documents[i:i + batch_size]

            for doc in batch:
                try:
                    # Preparar documento para inserción
                    db_doc = {
                        'content': doc['content'],
                        'embedding': doc['embedding'],
                        'metadata': doc['metadata'],
                        'chunk_hash': doc['chunk_hash']
                    }

                    # Intentar insertar
                    response = self.client.table('mapstruct_documents').insert(db_doc).execute()
                    inserted += 1

                except Exception as e:
                    if 'duplicate key' in str(e).lower():
                        duplicates += 1
                    else:
                        errors += 1
                        print(f"❌ Error: {str(e)[:100]}")

        print(f"\n📊 Resultados de inserción:")
        print(f"✅ Insertados: {inserted}")
        print(f"⚠️ Duplicados omitidos: {duplicates}")
        print(f"❌ Errores: {errors}")

        return inserted

    def verify_storage(self):
        """Verifica que los documentos se almacenaron correctamente"""
        try:
            response = self.client.table('mapstruct_documents').select('count').execute()
            count = response.count
            print(f"✅ Total de documentos en la base de datos: {count}")
            return count
        except Exception as e:
            print(f"❌ Error verificando almacenamiento: {str(e)}")
            return 0

    def test_similarity_search(self, query: str, model):
        """Prueba la búsqueda por similitud"""
        print(f"\n🔍 Probando búsqueda: '{query}'")

        # Generar embedding para la consulta
        query_embedding = model.encode(query, normalize_embeddings=True).tolist()

        # Buscar documentos similares
        response = self.client.rpc('match_mapstruct_documents', {
            'query_embedding': query_embedding,
            'match_threshold': 0.5,
            'match_count': 3
        }).execute()

        if response.data:
            print(f"✅ Encontrados {len(response.data)} resultados relevantes:")
            for i, result in enumerate(response.data, 1):
                print(f"\n{i}. Similitud: {result['similarity']:.3f}")
                print(f"   Contenido: {result['content'][:200]}...")
        else:
            print("❌ No se encontraron resultados")

        return response.data

# Inicializar store
vector_store = SupabaseVectorStore(SUPABASE_URL, SUPABASE_KEY)

✅ Conectado a Supabase


Almacenar documentos en Supabase


In [11]:
# Celda 11: Almacenar documentos en Supabase
# Insertar documentos
inserted_count = vector_store.insert_documents(chunks_with_embeddings, batch_size=50)

# Verificar almacenamiento
total_in_db = vector_store.verify_storage()

🔄 Insertando 1027 documentos en Supabase...


Insertando: 100%|██████████| 21/21 [05:03<00:00, 14.45s/it]



📊 Resultados de inserción:
✅ Insertados: 1026
⚠️ Duplicados omitidos: 1
❌ Errores: 0
✅ Total de documentos en la base de datos: None


Realizar pruebas de búsqueda

In [12]:
# Celda 12: Realizar pruebas de búsqueda
test_queries = [
    "How to map a list of DTOs in MapStruct?",
    "What are MapStruct annotations?",
    "Custom mapping methods",
    "Null value handling in MapStruct"
]

print("🧪 Ejecutando pruebas de búsqueda...")
print("=" * 50)

for query in test_queries:
    results = vector_store.test_similarity_search(query, model)
    print("-" * 50)

🧪 Ejecutando pruebas de búsqueda...

🔍 Probando búsqueda: 'How to map a list of DTOs in MapStruct?'
✅ Encontrados 1 resultados relevantes:

1. Similitud: 0.543
   Contenido: . All you have to do is to define a mapper interface which declares any required mapping methods. During compilation, MapStruct will generate an implementation of this interface. This implementation u...
--------------------------------------------------

🔍 Probando búsqueda: 'What are MapStruct annotations?'
✅ Encontrados 3 resultados relevantes:

1. Similitud: 0.648
   Contenido: The general philosophy of MapStruct is to generate code which looks as much as possible as if you had written it yourself from hand. In particular this means that the values are copied from source to ...

2. Similitud: 0.631
   Contenido: . i.e. they are not Collection or Map type properties. Collection-typed attributes with the same element type will be copied by creating a new instance of the target collection type containing the ele..

Generar estadísticas del proceso

In [13]:
# Celda 13: Generar estadísticas del proceso
def generate_statistics(chunks: List[Dict]):
    """Genera estadísticas útiles sobre el procesamiento"""

    stats = {
        'total_chunks': len(chunks),
        'total_characters': sum(len(c['content']) for c in chunks),
        'avg_chunk_size': np.mean([len(c['content']) for c in chunks]),
        'min_chunk_size': min(len(c['content']) for c in chunks),
        'max_chunk_size': max(len(c['content']) for c in chunks),
        'unique_sources': len(set(c['metadata']['source'] for c in chunks))
    }

    print("📊 Estadísticas del Procesamiento:")
    print("=" * 50)
    print(f"📄 Total de chunks: {stats['total_chunks']:,}")
    print(f"📝 Total de caracteres: {stats['total_characters']:,}")
    print(f"📐 Tamaño promedio de chunk: {stats['avg_chunk_size']:.0f} chars")
    print(f"📏 Rango de tamaños: {stats['min_chunk_size']} - {stats['max_chunk_size']} chars")
    print(f"📚 Fuentes únicas procesadas: {stats['unique_sources']}")

    # Distribución por fuente
    print("\n📑 Distribución por fuente:")
    sources = {}
    for chunk in chunks:
        source = chunk['metadata']['source']
        sources[source] = sources.get(source, 0) + 1

    for source, count in sources.items():
        percentage = (count / stats['total_chunks']) * 100
        print(f"  • {source}: {count} chunks ({percentage:.1f}%)")

    return stats

# Generar estadísticas
stats = generate_statistics(chunks_with_embeddings)

📊 Estadísticas del Procesamiento:
📄 Total de chunks: 1,027
📝 Total de caracteres: 891,794
📐 Tamaño promedio de chunk: 868 chars
📏 Rango de tamaños: 131 - 1000 chars
📚 Fuentes únicas procesadas: 2

📑 Distribución por fuente:
  • mapstruct-reference-guide.pdf: 234 chunks (22.8%)
  • ASO  Platform Documentation 2025.txt: 793 chunks (77.2%)


Resumen final y exportar configuración

In [14]:
# Celda 14: Resumen final y exportar configuración
config_summary = {
    'supabase_url': SUPABASE_URL,
    'table_name': 'mapstruct_documents',
    'embedding_model': 'all-MiniLM-L6-v2',
    'embedding_dimensions': 384,
    'chunk_size': processor.chunk_size,
    'chunk_overlap': processor.chunk_overlap,
    'total_documents_processed': len(uploaded_docs),
    'total_chunks_created': stats['total_chunks'],
    'total_chunks_stored': total_in_db,
    'processing_date': datetime.now().isoformat()
}

print("✅ PROCESO COMPLETADO EXITOSAMENTE")
print("=" * 50)
print("\n📋 Configuración para usar en el backend Lambda:")
print(json.dumps(config_summary, indent=2))

print("\n🚀 Próximos pasos:")
print("1. ✅ Base de conocimientos creada en Supabase")
print("2. ⏭️ Configurar función AWS Lambda con estos parámetros")
print("3. ⏭️ Desarrollar el plugin de VS Code")
print("4. ⏭️ Realizar pruebas de integración")

# Guardar configuración como archivo JSON
config_filename = 'rag_config.json'
with open(config_filename, 'w') as f:
    json.dump(config_summary, f, indent=2)

# Descargar archivo de configuración
files.download(config_filename)
print(f"\n💾 Archivo de configuración '{config_filename}' descargado")

✅ PROCESO COMPLETADO EXITOSAMENTE

📋 Configuración para usar en el backend Lambda:
{
  "supabase_url": "https://jqhmaqjzszaqvlvrpfpq.supabase.co",
  "table_name": "mapstruct_documents",
  "embedding_model": "all-MiniLM-L6-v2",
  "embedding_dimensions": 384,
  "chunk_size": 1000,
  "chunk_overlap": 200,
  "total_documents_processed": 2,
  "total_chunks_created": 1027,
  "total_chunks_stored": null,
  "processing_date": "2025-09-25T00:04:12.264896"
}

🚀 Próximos pasos:
1. ✅ Base de conocimientos creada en Supabase
2. ⏭️ Configurar función AWS Lambda con estos parámetros
3. ⏭️ Desarrollar el plugin de VS Code
4. ⏭️ Realizar pruebas de integración


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


💾 Archivo de configuración 'rag_config.json' descargado


Funciones auxiliares para mantenimiento futuro


In [15]:
# Celda 15: Funciones auxiliares para mantenimiento futuro
def update_knowledge_base(new_pdf_path: str):
    """Función para actualizar la base de conocimientos con nuevos documentos"""
    print(f"📤 Actualizando base de conocimientos con: {new_pdf_path}")
    # Implementar lógica de actualización incremental
    pass

def clear_database():
    """Limpia toda la base de datos (usar con precaución)"""
    confirm = input("⚠️ ¿Estás seguro de que quieres limpiar toda la base de datos? (yes/no): ")
    if confirm.lower() == 'yes':
        try:
            response = vector_store.client.table('mapstruct_documents').delete().neq('id', 0).execute()
            print("✅ Base de datos limpiada")
        except Exception as e:
            print(f"❌ Error: {e}")
    else:
        print("❌ Operación cancelada")

def export_chunks_to_json():
    """Exporta todos los chunks a un archivo JSON para backup"""
    filename = f'chunks_backup_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'

    # Remover embeddings para reducir tamaño
    chunks_no_embeddings = []
    for chunk in chunks_with_embeddings:
        chunk_copy = chunk.copy()
        chunk_copy.pop('embedding', None)
        chunks_no_embeddings.append(chunk_copy)

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(chunks_no_embeddings, f, ensure_ascii=False, indent=2)

    files.download(filename)
    print(f"✅ Backup exportado: {filename}")

print("🔧 Funciones de mantenimiento disponibles:")
print("  • update_knowledge_base() - Actualizar con nuevos documentos")
print("  • clear_database() - Limpiar base de datos")
print("  • export_chunks_to_json() - Exportar backup de chunks")

🔧 Funciones de mantenimiento disponibles:
  • update_knowledge_base() - Actualizar con nuevos documentos
  • clear_database() - Limpiar base de datos
  • export_chunks_to_json() - Exportar backup de chunks


In [16]:
export_chunks_to_json()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Backup exportado: chunks_backup_20250925_000741.json


## 📚 Documentación Adicional

### Modelo de Embeddings Utilizado
 - **Modelo**: `all-MiniLM-L6-v2`
 - **Dimensiones**: 384
 - **Ventajas**: Rápido, eficiente, excelente para búsqueda semántica
 - **Tamaño**: ~80MB
 - **Velocidad**: ~14,200 oraciones/segundo en GPU
### Estrategia de Chunking
 - **Tamaño**: 1000 caracteres (óptimo para contexto)
 - **Overlap**: 200 caracteres (preserva contexto entre chunks)
 - **Separadores**: Jerárquicos (párrafos > oraciones > palabras)
### Optimizaciones Implementadas
 1. **Deduplicación**: Hash SHA-256 previene duplicados
 2. **Batch Processing**: Mejora velocidad 10x
 3. **Normalización**: Embeddings normalizados para cosine similarity
 4. **Metadatos enriquecidos**: Facilita filtrado y debugging

### Troubleshooting Común

| Problema | Solución |
|----------|----------|
| Error de conexión a Supabase | Verificar URL y KEY en secretos |
| Chunks muy grandes/pequeños | Ajustar chunk_size en DocumentProcessor |
| Búsquedas sin resultados | Reducir match_threshold (0.5 → 0.3) |
| Duplicados en inserción | Normal, el sistema los maneja automáticamente |